<a href="https://colab.research.google.com/github/tuhinmallick/AI-for-Fashion/blob/main/Fine_tune_Gemma_2_on_Your_Computer_With_Transformers_and_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


*More details in this article: [Fine-tune Gemma 2 on Your Computer with LoRA and QLoRA](https://newsletter.kaitchup.com/p/fine-tune-gemma-2-on-your-computer)*

This notebook shows how to fine-tune Gemma 2 with QLoRA and LoRA. It works on a 24 GB GPU with Transformers or a 16 GB GPU with Unsloth.

The notebook is organized in 3 parts:
* QLoRA fine-tuning with Transformers
* LoRA fine-tuning with Transformers
* QLoRA fine-tuning with Unsloth

I don't provide code LoRA fine-tuning with Unsloth as I got errors when I wrote this notebook.

#With Transformers

We will need the following packages:


In [ ]:
!pip install --upgrade bitsandbytes transformers peft accelerate datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    

## QLoRA Fine-tuning

The code is very similar to the one I used for fine-tuning Llama 3, except that we don't need to configure a pad token.

[Fine-tune Llama 3 on Your Computer](https://kaitchup.substack.com/p/fine-tune-llama-3-on-your-computer)

In [ ]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from trl import SFTTrainer, SFTConfig

if torch.cuda.is_bf16_supported():
  os.system('pip install flash_attn')
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

model_name = "google/gemma-2-9b"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.padding_side = 'right'

ds = load_dataset("timdettmers/openassistant-guanaco")



bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model, gradient_checkpointing_kwargs={'use_reentrant':True})

print(model)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.00,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)


training_arguments = SFTConfig(
        output_dir="./gemma-2-9b/r16a16_QLoRA",
        eval_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=20,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=20,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        dataset_text_field="text",
        max_seq_length=512,
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Repo card metadata block was not found. Setting CardData to empty.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=12):   0%|          | 0/9846 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/518 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

Step,Training Loss,Validation Loss
20,1.541400,1.430857
40,1.358600,1.357276
60,1.304500,1.346512
80,1.331200,1.341357
100,1.310900,1.338429
120,1.301900,1.336290
140,1.307500,1.333460
160,1.296900,1.332452
180,1.304800,1.330233
200,1.319600,1.328650



***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4


Step,Training Loss,Validation Loss
20,1.541400,1.430857
40,1.358600,1.357276
60,1.304500,1.346512
80,1.331200,1.341357
100,1.310900,1.338429
120,1.301900,1.336290
140,1.307500,1.333460
160,1.296900,1.332452
180,1.304800,1.330233
200,1.319600,1.328650



***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4
Saving model checkpoint to ./gemma-2-9b/r16a16_QLoRA/checkpoint-307
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-9b/snapshots/beb0c08e9eeb0548f3aca2ac870792825c357b7d/config.json
Model config Gemma2Config {
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3584,
  "init

TrainOutput(global_step=307, training_loss=1.322845154554137, metrics={'train_runtime': 13497.883, 'train_samples_per_second': 0.729, 'train_steps_per_second': 0.023, 'total_flos': 2.2335744778621747e+17, 'train_loss': 1.322845154554137, 'epoch': 0.9975629569455727})

##LoRA Fine-tuning

In [ ]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
from trl import SFTTrainer, SFTConfig

if torch.cuda.is_bf16_supported():
  os.system('pip install flash_attn')
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

model_name = "google/gemma-2-9b"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.padding_side = 'right'

ds = load_dataset("timdettmers/openassistant-guanaco")

#Add the EOS token
def process(row):
    row["text"] = row["text"]+tokenizer.eos_token
    return row

ds = ds.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)


model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, torch_dtype=compute_dtype, attn_implementation=attn_implementation
)

model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={'use_reentrant':True})

print(model)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.00,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

training_arguments = SFTConfig(
        output_dir="./gemma-2-9b/r16a16_LoRA",
        eval_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=32,
        per_device_eval_batch_size=1,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=20,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=20,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        dataset_text_field="text",
        max_seq_length=512,
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Repo card metadata block was not found. Setting CardData to empty.
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=12):   0%|          | 0/9846 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/518 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm()
        (post_attention_layernorm): Gemma2RMSNorm()
        (pre_feedforwa

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Using auto half precision backend
Currently training with a batch size of: 1
***** Running training *****
  Num examples = 9,846
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 32
  Total optimization steps = 307
  Number of trainable parameters = 54,018,048
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
20,1.705400,1.538012
40,1.442500,1.446113
60,1.380900,1.433512
80,1.395200,1.428450
100,1.400700,1.424349
120,1.379500,1.423465
140,1.378000,1.421013
160,1.375500,1.417949
180,1.379300,1.416622
200,1.374400,1.414420



***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evaluation *****
  Num examples = 518
  Batch size = 1

***** Running Evalu

TrainOutput(global_step=307, training_loss=1.4054545517464803, metrics={'train_runtime': 8449.164, 'train_samples_per_second': 1.165, 'train_steps_per_second': 0.036, 'total_flos': 1.441453151470725e+17, 'train_loss': 1.4054545517464803, 'epoch': 0.9977655900873451})

#Unsloth

Install Unsloth and its dependencies. The following is what works with the current Colab configuration. It will change very often. Follow the [installation instructions provided in the GitHub](https://github.com/unslothai/unsloth) if it doesn't work.

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ntfb2hp1/unsloth_98f31e582da241fc93556f25610a3d2e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ntfb2hp1/unsloth_98f31e582da241fc93556f25610a3d2e
  Resolved https://github.com/unslothai/unsloth.git to commit 92dce38e8b3c1db209cef860d90b60188e95f0f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta

##QLoRA


In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
import multiprocessing
import torch
max_seq_length = 512
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

ds = load_dataset("timdettmers/openassistant-guanaco")

#Add the EOS token
def process(row):
    row["text"] = row["text"]+tokenizer.eos_token
    return row

ds = ds.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)


training_arguments = SFTConfig(
        output_dir="./gemma-2-9b/r16a16_QLoRA_Unsloth",
        eval_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=20,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=20,
        num_train_epochs=1,
        warmup_ratio=0.1,
        dataset_text_field="text",
        max_seq_length=512,
        lr_scheduler_type="linear",
)



trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Gemma2 patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.7 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.
Repo card metadata block was not found. Setting CardData to empty.


Map (num_proc=12):   0%|          | 0/9846 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/518 [00:00<?, ? examples/s]

Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Using auto half precision backend
Currently training with a batch size of: 4
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,846 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 307
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss,Validation Loss
20,1.541100,1.429181
40,1.358000,1.356442
60,1.303600,1.345572
80,1.330300,1.340242
100,1.310000,1.337499
120,1.301000,1.335561
140,1.306600,1.333249
160,1.296600,1.331671
180,1.304200,1.329507
200,1.318900,1.327860



***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evaluation *****
  Num examples = 518
  Batch size = 4

***** Running Evalu

KeyError: None